<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/LSTM-GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True)

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(LSTM(100,input_shape=(len(x_data.columns),1), activation='tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(Dense(100, activation='ReLU'))
model.add(GRU(100, input_shape=(100,1), activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 100)            40800     
                                                                 
 dense (Dense)               (None, 4, 100)            10100     
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense_1 (Dense)             (None, 4)                 404       
                                                                 
Total params: 111,904
Trainable params: 111,904
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_x, train_y, epochs=6, batch_size=32)

Epoch 1/6
414237/414237 [==============================] - 3067s 7ms/step - loss: 0.0325 - accuracy: 0.9891
Epoch 2/6
414237/414237 [==============================] - 3206s 8ms/step - loss: 0.0251 - accuracy: 0.9914
Epoch 3/6
414237/414237 [==============================] - 3277s 8ms/step - loss: 0.0243 - accuracy: 0.9917
Epoch 4/6
414237/414237 [==============================] - 3366s 8ms/step - loss: 0.0239 - accuracy: 0.9919
Epoch 5/6
414237/414237 [==============================] - 3370s 8ms/step - loss: 0.0229 - accuracy: 0.9920
Epoch 6/6
414237/414237 [==============================] - 3299s 8ms/step - loss: 0.0175 - accuracy: 0.9943


In [11]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 291s 3ms/step


In [12]:
confusion_matrix(test_y, pred_y)

array([[ 117727,       0,       0,       0],
       [      0,   96132,    2070,       0],
       [  20228,     172, 2827510,       0],
       [      0,       0,       0,  250056]])

In [13]:
report=classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.85337   1.00000   0.92089    117727
           1    0.99821   0.97892   0.98847     98202
           2    0.99927   0.99284   0.99604   2847910
           3    1.00000   1.00000   1.00000    250056

    accuracy                        0.99322   3313895
   macro avg    0.96271   0.99294   0.97635   3313895
weighted avg    0.99411   0.99322   0.99345   3313895

